# Count up reads in peaks, subset cell-type specific and link peaks to genes

**Authorship:**
Author, *MM/DD/YYYY*
***
**Description:**
Notebook to do some cool stuff
***
**TODOs:**
 - <font color='red'> count up reads in peaks </font>
 - <font color='red'> subset to cell-type specific objects per sample </font>
 - <font color='red'> link peaks to genes </font>
***

# Set-up

In [1]:
# Load libraries, takes between 20s and 1 min
suppressMessages(library(Signac))
suppressMessages(library(Seurat))
suppressMessages(library(EnsDb.Hsapiens.v86))
suppressMessages(library(BSgenome.Hsapiens.UCSC.hg38))
suppressMessages(library(harmony))
suppressMessages(library(SeuratDisk))
suppressMessages(library(ggplot2))
options(future.globals.maxSize = 4000 * 1024^2)
warnLevel <- getOption('warn')
options(warn = -1)
set.seed(1234)

In [2]:
# Load in genomic annotations from the interwebs. Takes about 5 mins
print("Loading hg38 genome annotations from Ensembl")
annotations <- GetGRangesFromEnsDb(ensdb=EnsDb.Hsapiens.v86)
seqlevelsStyle(annotations) <- 'UCSC'
genome(annotations) <- 'hg38'

[1] "Loading hg38 genome annotations from Ensembl"


Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

# Load all needed data

## SCT assay for sample
Only need the SCT assay to perform correlation with counts

In [23]:
# Set-up sample and cell-type, will be arguments in script
sample <- "R207"
cell.type <- "delta"
print(sprintf("Creating CRE-gene links for %s cells in sample %s", cell.type, sample))

[1] "Creating CRE-gene links for delta in sample R207"


In [24]:
# Load in the Seurat object
path = file.path("../indv_sample_analysis", sprintf("%s.indv.analysis.h5seurat", sample))
print(sprintf("Loading from %s", path))
adata <- LoadH5Seurat(path, assays = c("SCT"), reductions = c("pca"), graphs = FALSE, neighbors = FALSE, verbose = FALSE)
table(adata$predicted.id)

[1] "Loading from ../indv_sample_analysis/R207.indv.analysis.h5seurat"


Validating h5Seurat file




            acinar activated_stellate              alpha               beta 
               655                 17               3072               4141 
             delta             ductal        endothelial              gamma 
               409                 76                 22                166 
        macrophage               mast quiescent_stellate 
                30                 17                  6 

In [25]:
# Grab cell-type ideas
ids <- colnames(adata)[adata$predicted.id == cell.type]

[1] "AAACCAACAAGTGAAC-1" "AAACCGGCAATCCTGA-1" "AAACCGGCATCACTTC-1"
  [4] "AAACGCGCAAGCTACC-1" "AAAGCTTGTGATTTGG-1" "AAAGCTTGTGTTGTGA-1"
  [7] "AAAGGTTAGCTATTGA-1" "AACAAGCCACACAATT-1" "AACATTGTCAAGTGTC-1"
 [10] "AACCTAATCATTACGA-1" "AACCTAATCGTCCTTA-1" "AACCTCACAAGGTCGA-1"
 [13] "AACGCCCAGGCATTGT-1" "AACGCTAGTGCGCATG-1" "AACTTAGTCCTTAAGA-1"
 [16] "AAGACATAGATGGAGC-1" "AAGACATAGCAACATC-1" "AAGCATGAGAACAAGT-1"
 [19] "AAGCCTGTCAATGAGG-1" "AAGCCTGTCTTTGACT-1" "AAGCGGGTCCTTGCAC-1"
 [22] "AAGCGTTTCCGGGACT-1" "AAGGATTAGCTAATTG-1" "AAGTTTGTCGCAAACT-1"
 [25] "AATATCCCAGGCCTTG-1" "AATATCCCAGTTATGT-1" "AATCCTAAGGCCTAAT-1"
 [28] "AATTTGCCAGTTGCGT-1" "ACAAAGGTCAGTGAAG-1" "ACAACAACAGTAAGTA-1"
 [31] "ACAACACTCCCGCAAA-1" "ACAGGATCACGTTACA-1" "ACATCAATCCGCTAGA-1"
 [34] "ACATCAATCTATCGCC-1" "ACCCAACCACATACTG-1" "ACCCGCTGTTTGACCT-1"
 [37] "ACCCGTAAGTCACTCC-1" "ACCTAAGGTGCTGGTG-1" "ACCTGTTGTTGGTTGA-1"
 [40] "ACCTTGTGTATTCGTC-1" "ACGACTTAGCAATAGG-1" "ACGCAAGCACCTGCCT-1"
 [43] "ACGGGAAGTGTCCTGC-1" "ACGTCAAGTCACAAAT-1" "ACGTCCAAGTCCTTCA-1"
 [46] "ACGTCCAAGTTAGTGC-1" "ACGTTACAGGTATTGC-1" "ACTAAAGCAGGCTAAG-1"
 [49] "ACTTCGCTCCTACCTA-1" "ACTTGAATCAAGCTTA-1" "ACTTTGTTCCATTACT-1"
 [52] "AGAACAAGTGCAATAT-1" "AGAGATTAGGGCCATC-1" "AGAGGAACATGTCGCG-1"
 [55] "AGCAACAAGTCCTTCA-1" "AGCGGACCAGTAAAGC-1" "AGCTACTAGGACGTAA-1"
 [58] "AGCTTGGTCATGCGTG-1" "AGCTTTAAGATTGAGG-1" "AGGAACCAGATGGAGC-1"
 [61] "AGGAACGGTACCTTAC-1" "AGGACGTAGGACCGCT-1" "AGGATCCGTGTTGTAG-1"
 [64] "AGGCAGGTCTGGCTTC-1" "AGGCTAAAGGTCGATT-1" "AGGTTAGAGCCTAACG-1"
 [67] "AGGTTTGGTCGCAATA-1" "AGTCCTGAGGAGTAAT-1" "AGTGGACAGCTTATGA-1"
 [70] "AGTTGTCAGCTATGAC-1" "ATAAGTCGTCGTTATC-1" "ATAAGTCGTTGCGCGA-1"
 [73] "ATACCGGTCGCACACA-1" "ATACCTGCACTTAGGC-1" "ATATGCATCGGCTATG-1"
 [76] "ATATGCTCAGAAATTG-1" "ATATGGTGTTCCGGCT-1" "ATATGGTGTTGTAAAC-1"
 [79] "ATCAAGACACCCACAG-1" "ATCATCAAGCACGATT-1" "ATCCATAAGCTGGCTA-1"
 [82] "ATCCCGCGTCCTAAGA-1" "ATCGAGGCATCCCGCT-1" "ATGAAGTAGAGGGACT-1"
 [85] "ATGACAACAGTAGGAT-1" "ATGACTCAGAAGGTGC-1" "ATGAGCCGTTAGACCA-1"
 [88] "ATGGTCTAGGTCTTGG-1" "ATGTTGTCACAGAACG-1" "ATTGAAGCATAGACCC-1"
 [91] "ATTGCGCCATTATGCG-1" "ATTGGTTCATCTTGAG-1" "ATTGTGATCGAGGAAC-1"
 [94] "CAAACACTCGTTAGTA-1" "CAAATCATCCTAAGTA-1" "CAATAGCTCAAACACC-1"
 [97] "CAATCCCTCTTGACCC-1" "CAATCTAAGAGGAGTC-1" "CAATCTAAGCCTGGTA-1"
[100] "CACAATATCTGTGCAG-1" "CACATAATCGAGGAGT-1" "CACCAACCAATATGGA-1"
[103] "CACCAACCAGAGGCTA-1" "CAGATTCAGTAACTCA-1" "CAGCCTAAGTCCTTCA-1"
[106] "CAGGCTATCGGGATTT-1" "CAGGGCTTCACGAATC-1" "CATAAGCTCGTTAAGC-1"
[109] "CATAGACTCATCACTT-1" "CATAGCTAGGGACTAA-1" "CATCACACAGCACGTT-1"
[112] "CATGCATAGGCCTTAG-1" "CATGGATTCCAGCACA-1" "CATGGCGGTTAGCGTA-1"
[115] "CATTCATTCCTAATGA-1" "CATTGTGCAACGTGCT-1" "CATTTGTTCTGTGCCT-1"
[118] "CCAAACCCAAGGTCCT-1" "CCATAAGGTAACGGGA-1" "CCATCATGTCATGAGC-1"
[121] "CCCATAACACTTAACG-1" "CCCTAATGTGACCTGG-1" "CCCTAATGTGTTCCCA-1"
[124] "CCCTCAGTCGAAGTGA-1" "CCGCAAATCTAACCAA-1" "CCGCACACATGAAATG-1"
[127] "CCGCCAACACCTCAGG-1" "CCGCCAACACTTACAG-1" "CCGCTAAAGGCTCATG-1"
[130] "CCGTTATGTCTTTGAC-1" "CCTCAATGTTTAGTCC-1" "CCTCAGTTCCATAATG-1"
[133] "CCTCAGTTCTGGCTTC-1" "CCTGTAACAGAAATGC-1" "CCTTATGTCGCATCCT-1"
[136] "CCTTCGGTCAAACCGT-1" "CCTTGCGTCCTTGCAC-1" "CGAACAAAGCGCCTAA-1"
[139] "CGAAGTAAGCGGATTT-1" "CGAATCTAGCTTTGGG-1" "CGACAAGCATGCTATG-1"
[142] "CGACCTGCATGTTTGG-1" "CGAGCGAAGAACCTGT-1" "CGAGGAAGTCTCACTG-1"
[145] "CGATATTCACTGACTA-1" "CGATTGAGTGTAATAC-1" "CGCATATAGTAAGTGG-1"
[148] "CGCCACACAGCAACAG-1" "CGCTATGAGAGGAGTC-1" "CGCTCCATCGCATTAA-1"
[151] "CGGACCTAGGGACCTC-1" "CGGCCATAGATGGACA-1" "CGGCTCACAGGCCATT-1"
[154] "CGGGCTTAGTCTATGA-1" "CGGTGAACATAAGCAA-1" "CGTAACTAGGCCTTAG-1"
[157] "CGTACGGGTGATGGCT-1" "CGTTAGTAGTATTGGC-1" "CGTTATTTCACAGGAA-1"
[160] "CGTTATTTCGATATTG-1" "CGTTATTTCGTTTCGC-1" "CGTTTCTCATTAAGCT-1"
[163] "CTAATAGTCATTACTC-1" "CTACTAAAGGACCAGG-1" "CTAGCTGAGTGCACGC-1"
[166] "CTAGCTGAGTTAGAGG-1" "CTAGTCGAGCTCAATA-1" "CTATGATCATTAAGCT-1"
[169] "CTATGATCATTGCGTA-1" "CTCAATAAGTCACCAG-1" "CTCAGGATCTGGCAAT-1"
[172] "CTCATCATCGCTAAGT-1" "CTCATCATCGTGCTAG-1" "CTCATTAGTAACGGGA-1"


## Peaks for 3 cell types of interest
Loaded from previous MACS peak calling on all cell-types

In [26]:
# Create a granges object from bed file
file = file.path(sprintf("../macs_peaks_merged/%s_peaks.narrowPeak", cell.type))
df <- read.table(file, header = FALSE)

colnames(df) <- c("chr", "start", "end", "name", "score", 
                  "strand", "fold_change", "neg_log10pvalue_summit",
                  "neg_log10qvalue_summit", "relative_summit_position")

gr <- makeGRangesFromDataFrame(df, keep.extra.columns = TRUE)
gr <- keepStandardChromosomes(gr , pruning.mode = "coarse")
gr <- subsetByOverlaps(x = gr, ranges = blacklist_hg38_unified, invert = TRUE)
gr

GRanges object with 65569 ranges and 6 metadata columns:
          seqnames            ranges strand |             name     score
             <Rle>         <IRanges>  <Rle> |      <character> <integer>
      [1]     chr1        9931-10455      * |    delta_peak_27       209
      [2]     chr1     180720-181617      * |    delta_peak_28       302
      [3]     chr1     190715-191881      * |    delta_peak_29       431
      [4]     chr1     267846-268100      * |    delta_peak_30       164
      [5]     chr1     586054-586265      * |    delta_peak_31        92
      ...      ...               ...    ... .              ...       ...
  [65565]     chrY 19076872-19077293      * | delta_peak_65685        42
  [65566]     chrY 19566884-19567905      * | delta_peak_65686       166
  [65567]     chrY 19744397-19744833      * | delta_peak_65687        56
  [65568]     chrY 20575474-20576123      * | delta_peak_65688       169
  [65569]     chrY 26670911-26671157      * | delta_peak_65689     

# Find counts in peaks and create ChromatinAssay object

In [27]:
cells <- subset(x = adata, idents = cell.type)

In [28]:
# First create a fragments object
fragments <- CreateFragmentObject(
    path = file.path(sprintf("../%s", sample), "atac_fragments.tsv.gz"),
    cells = ids,
    validate.fragments = TRUE)

Computing hash



In [29]:
# For parallelization of FeatureMatrix command
suppressMessages(library(future))
plan("multicore", workers = 16)  # Change based on availability, same for below

In [30]:
# Get the counts in fragments and store as a sparse matrix
counts <- FeatureMatrix(
    fragments = fragments,
    features = gr,
    cells = ids,
    process_n = 10000000
)

Extracting reads overlapping genomic regions



In [42]:
# create a new assay using the MACS2 peak set and add it to the Seurat object
cells[["peaks"]] <- CreateChromatinAssay(
    counts = counts,
    fragments = fragments,
    annotation = annotations,
    ranges = gr,
    genome = "hg38"
)

In [43]:
cells

An object of class Seurat 
89819 features across 409 samples within 2 assays 
Active assay: peaks (65569 features, 0 variable features)
 1 other assay present: SCT
 1 dimensional reduction calculated: pca

# Link peaks to genes for this cell-type

In [33]:
# Alpha cells
DefaultAssay(cells) <- "peaks"
cells <- RegionStats(cells, genome = BSgenome.Hsapiens.UCSC.hg38)

In [38]:
# link peaks to genes. Adjust cut-offs so that all gene peak links are kept and non are discarded
cells <- LinkPeaks(
    object = cells,
    peak.assay = "peaks",
    expression.assay = "SCT",
    genes.use = c("SST"),
    method="pearson",
    pvalue_cutoff = 2,
    score_cutoff = 0
)

Testing 1 genes and 50483 peaks



In [40]:
# Save links to file
gr.links <- Links(cells)
linked.peaks <- gr.links$peak
chr.start.end <- unlist(strsplit(linked.peaks, "-"))
chr.start.end <- matrix(chr.start.end, length(chr.start.end)/3, 3, byrow = T)
seqs <- chr.start.end[, 1]
starts <- chr.start.end[, 2]
ends <- chr.start.end[, 3]
links.df <- data.frame(
    peak=gr.links$peak,
    gene=gr.links$gene,
    score=score(gr.links),
    zscore=gr.links$zscore,
    pvalue=gr.links$pvalue)
gr.peaks <- granges(cells)
gr.peaks$peak <- paste0(seqnames(gr.peaks), "-", start(gr.peaks), "-", end(gr.peaks))
annnotated.links <- merge(gr.peaks, links.df, by="peak", suffixes=c("_peaks", "_links"))
write.table(annnotated.links, 
            file=file.path("../indv_sample_networks", "correlation_links_shared-peaks", sprintf("%s.%s.links.tsv", sample, cell.type)),
            quote=F, sep="\t", row.names=F, col.names=T)

In [41]:
# Save object to file
saveRDS(cells, file = file.path("../indv_sample_networks", "correlation_links_shared-peaks", 
                                      sprintf("%s.%s.linked.rds", sample, cell.type)))

# Scratch
Place for old or testing code

# References